In [1]:
import pandas as pd

In [41]:
basics = pd.read_csv('./data/imdb/title_basics.tsv', sep = "\t").dropna()

<ipython-input-41-8d7a6074a46a>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  basics = pd.read_csv('./data/imdb/title_basics.tsv', sep = "\t").dropna()


In [42]:
basics.primaryTitle = basics.primaryTitle.apply(lambda x: x.lower().strip())

<ipython-input-42-bd3a514a611f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basics.primaryTitle = basics.primaryTitle.apply(lambda x: x.lower().strip())


In [44]:
basics = basics.set_index("primaryTitle")["tconst"]

In [26]:
akas = pd.read_csv('./data/imdb/title_akas.tsv', sep = "\t").dropna()
ratings = pd.read_csv('./data/imdb/title_ratings.tsv', sep = "\t")

<ipython-input-26-32fb714f1fa2>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  akas = pd.read_csv('./data/imdb/title_akas.tsv', sep = "\t").dropna()


In [30]:
akas_primary = akas[akas.attributes == "literal title"]
akas_primary.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
11,tt0000002,4,Der Clown und seine Hunde,DE,\N,\N,literal title,0
24,tt0000003,9,Armer Pierrot,DE,\N,\N,literal title,0
27,tt0000004,3,Ein gutes Glas Bier,DE,\N,\N,literal title,0
43,tt0000005,8,Schmiedszene,DE,\N,\N,literal title,0


In [31]:
akas_primary.shape

(12127, 8)

In [4]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1922
1,tt0000002,5.8,259
2,tt0000003,6.5,1734
3,tt0000004,5.6,174
4,tt0000005,6.2,2546


In [5]:
akas.title = akas.title.apply(lambda x: x.lower().strip())
akas = akas[["titleId", "title"]].set_index('title')
akas.head()

,titleId
title,
карменсіта,tt0000001
carmencita,tt0000001
carmencita - spanyol tánc,tt0000001
καρμενσίτα,tt0000001
карменсита,tt0000001


In [16]:
movie_meta_columns = ['movie_id',
                    'freebase_movie_id',
                    'movie_name',
                    'release_date',
                    'box_office_revenue',
                    'runtime',
                    'languages',
                    'countries',
                    'genres']

movie_metadata = pd.read_csv('./data/movie.metadata.tsv', sep='\t', header=None, names=movie_meta_columns)
movie_metadata.movie_name = movie_metadata.movie_name.apply(lambda x: x.lower().strip())
movie_metadata = movie_metadata[["movie_id", "movie_name"]].set_index("movie_name")

movie_metadata.head()

,movie_id
movie_name,
ghosts of mars,975900
getting away with murder: the jonbenét ramsey mystery,3196793
brun bitter,28463795
white of the eye,9363483
a woman in flames,261236


In [56]:
merge = pd.merge(movie_metadata, akas, left_index= True, right_index = True)
merge.head()

MemoryError: cannot allocate memory for array

In [ ]:
merge = merge.drop_duplicates(subset = 'movie_id')

In [ ]:
freebase_movies_imdb_score = pd.merge(merge.set_index("titleId"), ratings.set_index("tconst"), left_index=True, right_index=True)
freebase_movies_imdb_score = freebase_movies_imdb_score.reset_index().drop("index", axis = 1).set_index("movie_id")

In [ ]:
freebase_movies_imdb_score

,averageRating,numVotes
movie_id,,
7669437,5.7,1922
9996498,5.8,259
6431647,6.5,1734
9999926,5.6,174
73372,6.2,2546
...,...,...
24319366,5.4,52
18475018,5.8,6
7316179,6.0,6


In [ ]:
print(f"{movie_metadata.shape[0]-freebase_movies_imdb_score.shape[0]} \
freebase titles lost in merge (\
{100 * (movie_metadata.shape[0]-freebase_movies_imdb_score.shape[0])/movie_metadata.shape[0]:.3f})%")

24013 freebase titles lost in merge (29.377)%


In [55]:
saved = freebase_movies_imdb_score.copy()